In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
import qiskit
import ipyparallel as ipp
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.providers.aer import AerSimulator
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper,ParityMapper
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from qiskit.circuit.random import random_circuit
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator,Z2Symmetries
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy

In [2]:
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def egBandHamiltonianPartition(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_0=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                line=line.split()
                i,j=map(int,line[:2])
                val=float(line[2])
                if(i==j):
                    eg_h1_0[i,j]=eg_h1_0[i+num_sites,j+num_sites]=val
                else:
                    eg_h1_1[i,j]=eg_h1_1[i+num_sites,j+num_sites]=val
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1_0=eg_h1_0+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2_0=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        eg_h2_1=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            if((i==j) and (k==l)):
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k,l] = eg_h2_0[i,j,k+num_sites,l+num_sites] \
                = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
            else:
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
                eg_h2_1[i+num_sites,j+num_sites,k,l] = eg_h2_1[i,j,k+num_sites,l+num_sites] = 0.5*val
    return eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1
def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    E=(state.conj()@Hmat@state).real
    return E

In [3]:
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat
def commutatorPool(qubitH,stripZs=False):
    def stripZ(label):
        label=numpy.array(list(label))
        label[label=='Z']='I'
        label=''.join(list(label))
        return label
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels=numpy.unique(labels)
    if stripZs==True:
        new_labels=[]
        for i in range(len(unique_labels)):
            new_labels.append(stripZ(unique_labels[i]))
        unique_labels=numpy.unique(new_labels)    
    commutator_pool=[WeightedPauliOperator([[1j,Pauli(unique_labels[i])]]) for i in range(len(unique_labels))]
    return commutator_pool
def compute_gradient(op,params):
    var_form_base.push_hopping_operator(op)
    E1=Energy(params+[numpy.pi/4.])
    E2=Energy(params+[-numpy.pi/4.])
    gradient=(E1-E2)/2
    var_form_base.pop_hopping_operator()
    return gradient

In [50]:
#Adadelta with Statevector
backend=Aer.get_backend('statevector_simulator')
def Energy_Estimator(circ):
    state=execute(circ,backend,shots=1024).result().get_statevector()
    E=(state.conj()@Hmat@state).real
    return E
def AdaDelta_SV(cost,params,runs=20,save_opt_steps=False):
    v=numpy.zeros((len(params)))*0.0
    delta=numpy.zeros((len(params)))*0.0
    beta=0.9
    directions= numpy.arange(len(params))
    delta=0
    circ=var_form_base.construct_circuit(parameters=params)
    E_arr=[]
    ind=0
    eps=numpy.array([2e-11]*len(params))#1e-5,5e-6,1e-7,5e-8
    def Energy_Estimator(circ):
        state=execute(circ,backend,shots=1024).result().get_statevector()
        E=(state.conj()@Hmat@state).real
        return E
    def StochGrad(cost,params,directions):
        shiftedParams1=[numpy.array(params)+numpy.array([0]*ei+[numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        shiftedParams2=[numpy.array(params)+numpy.array([0]*ei+[-numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        stochGrads=numpy.zeros((len(params)))
        data_arr=[]
        for i in range(len(shiftedParams1)):
            circ=var_form_base.construct_circuit(parameters=shiftedParams1[i])
            data_arr=data_arr+[circ]
        for i in range(len(shiftedParams2)):
            circ=var_form_base.construct_circuit(parameters=shiftedParams2[i])
            data_arr=data_arr+[circ]   
        results=Parallel(n_jobs=6,verbose=2)(delayed(cost)(data_arr[i]) for i in range(len(data_arr)))    
        estm1Arr=[results[i] for i in range(0,len(results)//2)]#map(cost,shiftedParams1)
        estm2Arr=[results[i] for i in range(len(results)//2,len(results))]
        stochGrads=0.5*(numpy.array(list(estm1Arr))-numpy.array(list(estm2Arr)))
        return stochGrads
    flag=0
    flag1=0
    terminate=False
    for ind in range(runs):
        g_stoch=StochGrad(cost,params,directions)
        v = beta*v+(1-beta)*g_stoch*g_stoch
        lr=numpy.sqrt(delta+eps)/numpy.sqrt(v+eps)  #learning rate
        g=lr*g_stoch
        params=params-g
        params=params.real
        delta= beta*delta+(1-beta)*g*g
        print("learning rate",lr)
        E=Energy(params)
        print("Energy",E)
        if save_opt_steps==True:
            with open('OptStepsWithSV_parityU=7.txt','a') as f:
                print('['+','.join([str(params[i]) for i in range(len(params))])+']'+'#'+str(E),file=f)
        ind=ind+1 
        #print("error",conv_err)
    res={'x':params,'fun':E}    
    return res

In [51]:
#preparing commutator pool
U=7
h1_D,h1_X,h2_D,h2_X=egBandHamiltonianPartition(U)
def qubitWeightedOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('parity')
    qubitH=Z2Symmetries.two_qubit_reduction(qubit_op, qubit_op.num_qubits//2)
    return qubitH
qubitH=qubitWeightedOp(h1_D+h1_X,h2_D+h2_X)
Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
w,v=numpy.linalg.eigh(Hmat)
Eg=w[0]
commutator_pool=commutatorPool(qubitH,stripZs=False)
#constructing matrices for gradient computation
#MatrixRepOfPoolOps=list(map(commutatorsForGradient,commutator_pool))
#print("pool sizes",len(MatrixRepOfPoolOps))
#backend=Aer.get_backend('statevector_simulator')
#preparing init s>tate
circ=QuantumCircuit(6)
circ.x(0)
circ.x(3)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='parity',two_qubit_reduction=True)
var_form_base.manage_hopping_operators()
error=1000

In [52]:
Eg

-19.184985040869176

In [53]:
Energy([])

-18.111999999999988

In [54]:
import time

In [55]:
with open('OptStepsWithSV_parityU=7.txt','r') as f:
    lines=f.readlines()
    Labels=[]
    EnergyArr=[]
    for i in range(len(lines)):
        if lines[i][0]=='l':
            Labels.append(lines[i].split('-')[1][1:-1])
            if i!=0:
                
                EnergyArr.append(eval(lines[i-1].split('#')[1]))
    EnergyArr.append(eval(lines[-1].split('#')[1]))            
    params=eval(lines[-1])   
print(Labels)    
for i in range(len(Labels)):
    var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(Labels[i])]]))    

['ZYXIII', 'IIIXYI', 'YXZIII', 'IIIZXY', 'IIZIYX', 'IZZXYI', 'IZZZXY', 'IIZXYI', 'IYXIZZ', 'ZYIIIX', 'ZYIIZX', 'IIXXXY', 'IIXYXX', 'YXZIIZ', 'IIIIXY', 'IZXYXX', 'IIZXYI', 'YXXIIX', 'IIXIYZ', 'IZXXXY', 'IYZIZX', 'IIIIXY', 'IIXIIY', 'IXYIII', 'XYIIIZ', 'IZYIIX', 'IZXIIY', 'IIYIIX', 'IIXIIY', 'IIYIZX', 'IIXIIY', 'IZZXYI', 'IIXIIY']


In [56]:
EnergyArr

[-18.670874921554816,
 -18.72440014541459,
 -18.95632911611107,
 -18.978313789620223,
 -19.001030046394394,
 -19.026985736641446,
 -19.044749935785532,
 -19.088926507958824,
 -19.09762581350574,
 -19.10159032672683,
 -19.114708015305688,
 -19.119403665884665,
 -19.124151732199064,
 -19.129200662869223,
 -19.133014910859572,
 -19.134171074881287,
 -19.138638649254812,
 -19.14404990883495,
 -19.152126783560576,
 -19.158101264342832,
 -19.161625482293903,
 -19.175586128174096,
 -19.17933613770971,
 -19.180919866127752,
 -19.183038216441197,
 -19.18373071560808,
 -19.18404172972633,
 -19.184202523380954,
 -19.18433918099914,
 -19.184600591122358,
 -19.184712004261137,
 -19.184839138827297,
 -19.184867738407632]

In [57]:
res=AdaDelta_SV(Energy_Estimator,params,runs=400,save_opt_steps=True)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   10.6s


learning rate [0.0780461  0.06084607 0.06010916 0.10124807 0.1235151  0.03832963
 0.31065114 0.19974403 0.05366865 0.02178465 0.06400508 0.03130725
 0.01845532 0.33304902 0.09446451 0.02316752 0.13739812 0.02648499
 0.01389812 0.01787286 0.0272999  0.43412134 0.01556842 0.11542812
 0.12274186 0.02129058 0.01923779 0.01536003 0.01287051 0.020986
 0.01287051 0.14284561 0.01158997]


[Parallel(n_jobs=6)]: Done  66 out of  66 | elapsed:   16.3s finished


Energy -19.18486788603129


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    4.5s


...

[Parallel(n_jobs=6)]: Done  66 out of  66 | elapsed:   10.0s finished


Energy -19.1848682177181


Process ForkProcess-10970:
Process ForkProcess-10967:
Process ForkProcess-10969:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)


QiskitError: 'Keyboard interrupt in parallel_map.'

  File "/home/amukh/Anaconda3-Linux/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/queues.py", line 96, in get
    with self._rlock:
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/queues.py", line 96, in get
    with self._rlock:
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/queues.py", line 97, in get
    res = self._recv_bytes()
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/amukh/Anaconda3-Linux/lib/p

In [59]:
Eg+19.184839

-0.0001460408691755788

In [60]:
len(Labels)

33

In [44]:
import time
from tqdm import tqdm

In [ ]:
#params=[]
#EnergyArr=[]
ExcOps=Labels
ti=time.time()
steps=35
print_res=False
for i in range(35):
    grads=numpy.array(Parallel(n_jobs=1,verbose=2)(delayed(compute_gradient)(commutator_pool[i],params) for i in tqdm(range(len(commutator_pool)))))
    indexes=numpy.argsort(abs(grads))[::-1][:5]
    #Labels=[commutator_pool[indexes[i]].paulis[0][1].to_label() for i in range(len(indexes))]
    print("3 highest grads",grads[indexes[0]],grads[indexes[1]],grads[indexes[2]])
    print("3 highest grads",commutator_pool[indexes[0]].paulis[0][1].to_label(),commutator_pool[indexes[1]].paulis[0][1].to_label(),commutator_pool[indexes[2]].paulis[0][1].to_label())
    if len(ExcOps)!=0:
        if (commutator_pool[indexes[0]].paulis[0][1].to_label()!=ExcOps[-1]):
            PauliOp=commutator_pool[indexes[0]]
        else:
            PauliOp=commutator_pool[indexes[1]]
    else:
        PauliOp=commutator_pool[indexes[0]]
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("chosen Op",ExcOps[-1])
    with open('OptStepsWithSV_parityU=7.txt','a') as f:
        print("label-",ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    res=AdaDelta_SV(Energy_Estimator,params,runs=400,save_opt_steps=True)#SMO_qasm(hamiltonian_estimation_in_doubly_decomposed_form_qasm,params,runs=40,tol=1e-6,save_opt_steps=True)    #res=scipy.optimize.minimize(Energy_Estimator,params,method='BFGS',options={'ftol':1e-7})
    params,E=list(res['x']),res['fun']
    print("num_params",var_form_base.num_parameters)
    print("Energy",E)
    EnergyArr.append(E)
    print("time elapsed",time.time()-ti)    
    error=EnergyArr[-1]-Eg

  0%|          | 0/56 [00:00<?, ?it/s][Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.8s remaining:    0.0s
100%|██████████| 56/56 [01:53<00:00,  2.03s/it]

3 highest grads -0.013147221977089174 -0.011108557541136044 -0.010640646619465954
3 highest grads IIXIIY IIXIZY IIYIIX
chosen Op IIXIIY



[Parallel(n_jobs=1)]: Done  56 out of  56 | elapsed:  1.9min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   15.0s


learning rate [0.01642622 0.02382799 0.0147558  0.02508704 0.03651529 0.09658294
 0.0629179  0.09208992 0.01449454 0.00536028 0.00516683 0.00572611
 0.00467948 0.01511039 0.01134898 0.00512095 0.03457276 0.00604968
 0.00542738 0.00507426 0.00588293 0.02292257 0.00375262 0.04781436
 0.04143152 0.0058024  0.00506772 0.00433042 0.0035116  0.004838
 0.0035116  0.44617893 0.00340156]


[Parallel(n_jobs=6)]: Done  66 out of  66 | elapsed:   24.6s finished


...

Energy -19.18486762811827


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    7.2s


learning rate [0.1687682  0.20701026 0.15398308 0.22978541 0.36104721 0.13789866
 0.55338145 0.41652439 0.1098291  0.05217317 0.0330636  0.05404982
 0.04838233 0.09064808 0.11583851 0.04905393 0.20470242 0.06206818
 0.04048442 0.05376761 0.06010122 0.13971143 0.04065069 0.39290727
 0.23141909 0.04906941 0.05321141 0.04478988 0.03508055 0.05021544
 0.03508055 0.68420325 0.03180338]


[Parallel(n_jobs=6)]: Done  66 out of  66 | elapsed:   15.5s finished


Energy -19.18486774400295


In [ ]:
indexes=numpy.argsort(abs(grads))[::-1][:5]
label=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(indexes))]

In [ ]:
label

In [ ]:
grads